# Projeto 2 - Ciência dos Dados

**Nome: Breno Marti**

**Nome: João Pedro Chacon Ruiz**

Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
# Classificador automático de sentimento


## Preparando o ambiente no jupyter:

In [40]:
%%capture

#Instalando o tweepy
!pip install tweepy
!pip install emojis

In [3]:
import tweepy
import math
import os.path
import pandas as pd
import json
import numpy as np
from random import shuffle

___
## Autenticando no  Twitter

* Conta: ***[Preencha aqui o id da sua conta. Ex: @fulano ]***

In [4]:
#Dados de autenticação do twitter:

#Coloque aqui o identificador da conta no twitter: @fulano

#leitura do arquivo no formato JSON
with open('auth.pass') as fp:    
    data = json.load(fp)

#Configurando a biblioteca. Não modificar
auth = tweepy.OAuthHandler(data['consumer_key'], data['consumer_secret'])
auth.set_access_token(data['access_token'], data['access_token_secret'])

___
## Etapas do projeto:

### Escolha de um produto e coleta das mensagens


In [ ]:
#Produto escolhido:
produto = 'Bugatti'

#Quantidade mínima de mensagens capturadas:
n = 500
#Quantidade mínima de mensagens para a base de treinamento:
t = 300

#Filtro de língua, escolha uma na tabela ISO 639-1.
lang = 'pt'

Capturando os dados do twitter:

In [ ]:
#Cria um objeto para a captura
api = tweepy.API(auth)

#Inicia a captura, para mais detalhes: ver a documentação do tweepy
i = 1
msgs = []
for msg in tweepy.Cursor(api.search, q=produto, lang=lang, tweet_mode="extended").items():
    if (not msg.retweeted) and ('RT' not in msg.full_text):
        msgs.append(msg.full_text.lower())
        i += 1
    if i > n:
        break

#Embaralhando as mensagens para reduzir um possível viés
shuffle(msgs)

Salvando os dados em uma planilha Excel:

In [ ]:
#Verifica se o arquivo não existe para não substituir um conjunto pronto
if not os.path.isfile('./{0}.xlsx'.format(produto)):
    
    #Abre o arquivo para escrita
    writer = pd.ExcelWriter('{0}.xlsx'.format(produto))

    #divide o conjunto de mensagens em duas planilhas
    dft = pd.DataFrame({'Treinamento' : pd.Series(msgs[:t])})
    dft.to_excel(excel_writer = writer, sheet_name = 'Treinamento', index = False)

    dfc = pd.DataFrame({'Teste' : pd.Series(msgs[t:])})
    dfc.to_excel(excel_writer = writer, sheet_name = 'Teste', index = False)

    #fecha o arquivo
    writer.save()

___
### Classificando as mensagens na coragem

Esta etapa é manual. Faça a mesma pelo Excel.

___
### Montando o Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

In [5]:
raw = pd.read_excel('Bugatti_treino.xlsx')

In [6]:
relevante_treino_raw = raw[raw['Relevância'] == 1]

In [7]:
nao_relevante_treino_raw = raw[raw["Relevância"] == 0]

#### Funções de limpeza de texto:


In [26]:
import re 

def cleanup(text):
    """
        Função de limpeza muito simples que troca alguns sinais básicos por espaços
    """
    punctuation = '[#@,!\()-.":?;]' # Note que os sinais [] são delimitadores de um conjunto.
    pattern = re.compile(punctuation)
    text_subbed = re.sub(pattern, ' ', text)
    return text_subbed   

In [41]:
import emojis

In [42]:
def transform_emojis(text):
    clean = emojis.decode(text)
    return clean

#### Limpando e transformando os dataframes em séries para a análise:

In [45]:
list1 = []
for tweet in relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list1.append(cleanup(texto))
    
relevante_treino = pd.Series(list1)

In [46]:
relevante_treino

0     mds cade a koenigsegg pra bater esses 500km/h ...
1     bugatti bate recorde de velocidade com um carr...
2     depois de bater o próprio recorde de 490 km/h ...
3     podem não ser os mais bonitos  mas os carros d...
4     parece que teremos o bugatti chiron super spor...
5     bugatti descobriu o quão rápido o chiron é  fl...
6     bugatti chiron com velocidade final de 490 km/...
7     quase 500 kph em um carro  absurdo  https //t ...
8     tô pesquisando uns carros luxuosos aqui e perc...
9     após 490 km/h do chiron  bugatti anuncia que n...
10    bugatti chiron super sport 300   versão  civil...
11    bugatti ultrapassa a marca dos 480 km/h – e de...
12    bugatti quebra o recorde de velocidade com um ...
13    bugatti chiron chega a 490 km/h e bate recorde...
14    carro mais rápido do mundo  nova bugatti terá ...
15     williamenb outra coisa  porque aquele bugatti...
16    wow  a bugatti chiron chegou a 490 km/h    iss...
17    bugatti chiron  levemente modificada  cheg

In [51]:
l = ['\n']
relevante_treino = relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()

In [52]:
relevante_treino

0     mds cade a koenigsegg pra bater esses 500km/h ...
1     bugatti bate recorde de velocidade com um carr...
2     depois de bater o próprio recorde de 490 km/h ...
3     podem não ser os mais bonitos  mas os carros d...
4     parece que teremos o bugatti chiron super spor...
5     bugatti descobriu o quão rápido o chiron é  fl...
6     bugatti chiron com velocidade final de 490 km/...
7     quase 500 kph em um carro  absurdo  https //t ...
8     tô pesquisando uns carros luxuosos aqui e perc...
9     após 490 km/h do chiron  bugatti anuncia que n...
10    bugatti chiron super sport 300   versão  civil...
11    bugatti ultrapassa a marca dos 480 km/h – e de...
12    bugatti quebra o recorde de velocidade com um ...
13    bugatti chiron chega a 490 km/h e bate recorde...
14    carro mais rápido do mundo  nova bugatti terá ...
15    williamenb outra coisa  porque aquele bugatti ...
16    wow  a bugatti chiron chegou a 490 km/h    iss...
17    bugatti chiron  levemente modificada  cheg

In [48]:
list2 = []
for tweet in nao_relevante_treino_raw['Treinamento']:
    texto = transform_emojis(tweet)
    list2.append(cleanup(texto))
    
nao_relevante_treino = pd.Series(list2)

In [53]:
nao_relevante_treino

0      talla 42 eu  bugatti 3 11382e 11  botas clasic...
1      talla 40 eu  bugatti 3 11384e 11  botas clasic...
2       carolinaazered2  junior__bugatti com o bumbum...
3                fuga de bugatti com ela depois das três
4      bugatti verón supersport motor triturbo 8 0 l ...
5      el nuevo récord de velocidad del bugatti chiro...
6      se continuar esse calor vai todo mundo derrete...
7                                 lleobertodo menino bom
8      gente vamos jogar stop  não vale olhar na inte...
9      aberto até de madrugada  bugatti chiron chega ...
10     gente vamos jogar stop  não vale olhar na inte...
11      luaracaastro desse jeito  daqui a pouco volta...
12                    jenniferbraaga uma delicinha kkkkk
13                             paloma_souza18 foi ontem 
14      ballerichh claro q  lembro alugamos um  bugat...
15     mas eu conseguiria trazer um suco gástrico esp...
16              ofereçam me um bugatti  não peço mais xd
17      iamthebrunao tomara q n

In [54]:
nao_relevante_treino = nao_relevante_treino.str.replace('|'.join(l),'',regex=True).str.strip()

In [55]:
nao_relevante_treino

0      talla 42 eu  bugatti 3 11382e 11  botas clasic...
1      talla 40 eu  bugatti 3 11384e 11  botas clasic...
2      carolinaazered2  junior__bugatti com o bumbum ...
3                fuga de bugatti com ela depois das três
4      bugatti verón supersport motor triturbo 8 0 l ...
5      el nuevo récord de velocidad del bugatti chiro...
6      se continuar esse calor vai todo mundo derrete...
7                                 lleobertodo menino bom
8      gente vamos jogar stop  não vale olhar na inte...
9      aberto até de madrugada  bugatti chiron chega ...
10     gente vamos jogar stop  não vale olhar na inte...
11     luaracaastro desse jeito  daqui a pouco volta ...
12                    jenniferbraaga uma delicinha kkkkk
13                              paloma_souza18 foi ontem
14     ballerichh claro q  lembro alugamos um  bugatt...
15     mas eu conseguiria trazer um suco gástrico esp...
16              ofereçam me um bugatti  não peço mais xd
17     iamthebrunao tomara q nã

In [31]:
#serie_relevante_t = pd.Series(relevante_t.split())

In [14]:
#serie_nao_relevante_t = pd.Series(nao_relevante_t.split())

#### Contando o número de aparições de cada palavra :

In [15]:
#tabela_relevante_t = serie_relevante_t.value_counts()

In [16]:
#tabela_nao_relevante = serie_nao_relevante_t.value_counts()

#### Frequência relativa de cada palavra:

In [17]:
#tabela_relevante_t_relativa = serie_relevante_t.value_counts(True)

In [18]:
#tabela_nao_relevante_t_relativa = serie_nao_relevante_t.value_counts(True)

#### Soma de todas as palavras:

In [19]:
#soma_total_t = relevante_t + nao_relevante_t

___
### Verificando a performance

Agora você deve testar o seu classificador com a base de Testes.

___
### Concluindo

## Aperfeiçoamento:

Os trabalhos vão evoluir em conceito dependendo da quantidade de itens avançados:

* Limpar: \n, :, ", ', (, ), etc SEM remover emojis
* Corrigir separação de espaços entre palavras e emojis ou emojis e emojis
* Propor outras limpezas e transformações que não afetem a qualidade da informação ou classificação
* Criar categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante (3 categorias: C, mais categorias conta para B)
* Explicar por que não posso usar o próprio classificador para gerar mais amostras de treinamento
* Propor diferentes cenários para Naïve Bayes fora do contexto do projeto
* Sugerir e explicar melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* Montar um dashboard que periodicamente realiza análise de sentimento e visualiza estes dados

# Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**